In [1]:
import h5rdmtoolbox as h5tbx

In [2]:
from rdflib.namespace import FOAF
from rdflib import URIRef

In [3]:
from h5rdmtoolbox.convention.namespace import M4I, OBO

In [4]:
# from h5rdmtoolbox.ld.namespace import M4I
M4I.Method # http://w3id.org/nfdi4ing/metadata4ing#Method
M4I.NumericalVariable

rdflib.term.URIRef('http://w3id.org/nfdi4ing/metadata4ing#NumericalVariable')

In [11]:
with h5tbx.File() as h5:    
    
    mp = h5.create_group('MP')
    mp.iri = FOAF.Person
    mp.attrs['first name', FOAF.firstName] = 'Matthias'
    mp.attrs['last name', FOAF.lastName] = 'Probst'
    mp.attrs['orcid id'] = '123'

    piv_proc = h5.create_group('PIV_Pre')
    piv_proc.iri = M4I.Method
    piv_proc.attrs['has_participants', OBO.RO_0000057] = h5['MP']
    h5.dump(False)

In [14]:
with h5py.File(h5.hdf_filename) as h5p:
    print(h5p['MP'].file.filename)

C:\Users\da4323\AppData\Local\h5rdmtoolbox\h5rdmtoolbox\tmp\tmp_0\tmp1.hdf


In [20]:
import json
import h5py
from h5rdmtoolbox.wrapper.h5utils import get_rootparent
import pathlib


def json_dumps(grp, iri_only=False):
    base_url = 'http://hdf.file/'

    def _get_id(_grp):
        stem = pathlib.Path(_grp.file.filename).stem
        id = base_url + stem + _grp.name
        return id
        
    def _get_dict(_grp, _iri_only):
        j = {"@id": _get_id(_grp)}
        s = _grp.iri.subject
        if s is not None:
            j["@type"] = str(s)
        for k, v in _grp.attrs.items():
            if not k.isupper():
                if _grp.iri.predicate.get(k, None) is not None:
                    if isinstance(v, h5py.Group):
                        value = _get_id(v)
                    else:
                        value = str(v)
                    j[str(_grp.iri.predicate[k])] = value
                else:
                    if not _iri_only:
                        j[k] = str(v)
        return j

    if not isinstance(grp, (list, tuple)):
        grp = [grp, ]
        
    return json.dumps({"@graph": [_get_dict(g, iri_only) for g in grp]})


def json_dump(grp, fp, iri_only=False):
    base_url = 'http://hdf.file/'

    def _get_id(_grp):
        stem = pathlib.Path(_grp.file.filename).stem
        id = base_url + stem + _grp.name
        return id
        
    def _get_dict(_grp, _iri_only):
        j = {"@id": _get_id(_grp)}
        s = _grp.iri.subject
        if s is not None:
            j["@type"] = str(s)
        for k, v in _grp.attrs.items():
            if not k.isupper():
                if _grp.iri.predicate.get(k, None) is not None:
                    if isinstance(v, h5py.Group):
                        value = _get_id(v)
                    else:
                        value = str(v)
                    j[str(_grp.iri.predicate[k])] = value
                else:
                    if not _iri_only:
                        j[k] = str(v)
        return j

    if not isinstance(grp, (list, tuple)):
        grp = [grp, ]
        
    return json.dump({"@graph": [_get_dict(g, iri_only) for g in grp]}, fp, indent=4)
    
def jsonld_dump(grp):
    return json_dump(grp, True)

with h5tbx.File(h5.hdf_filename) as h5:
    with open('test.jsonld', 'w') as f:
        json_dump([h5['MP'], h5['PIV_Pre']], f)
    # print(jsonld_dump(h5['PIV_Pre']))

In [ ]:
from pprint import pprint

In [ ]:
FOAF.firstName

In [ ]:
CONTEXT = dict(
    m4i = "http://w3id.org/nfdi4ing/metadata4ing#",
    prov = "http://www.w3.org/ns/prov#",
    foaf = "http://xmlns.com/foaf/0.1/",
)
IRIdict = dict(
    method='m4i:Method',
    person='prov:Person',
    firstName='foaf:firstName'
)

for k, v in IRIdict.copy().items():
    for kk, vv in CONTEXT.items():
        if kk in v:
            IRIdict[k] = v.replace(f'{kk}:', vv)
    IRIdict[k] = IRI(IRIdict[k])
        
IRIdict

In [ ]:
FOAF.firstName.fragment

In [ ]:
FOAF.Person

In [ ]:
with h5tbx.File() as h5:
    mp = h5.create_group('MP')
    mp.attrs['first name', FOAF.firstName] = 'Matthias'
    h5.dump()